# Tests for the skill profiling framework

Run some tests on functions used in the skill profiling framework.

In [103]:
import pandas as pd
import numpy as np

# local imports
from config import data_dir
from analog_forecast import read_subset_era5
from run_profile import get_naive_sample_dates, profile_naive_forecast

Load a dataset (spatial subset) for doing some testing with.

In [2]:
%%time

varname = "t2m"
spatial_domain = "alaska"
ref_date = "2004-10-11"

raw_sub_da = read_subset_era5(spatial_domain, data_dir, varname, use_anom=False).load()

CPU times: user 4.98 s, sys: 9.69 s, total: 14.7 s
Wall time: 44.6 s


`get_naive_sample_dates` should return a tuple of ("list of naive analog dates and 14 dates for each" (`all_naive_dates` below), "list of naive analog dates" (`naive_analog_dates` below). Run some verifications.

In [104]:
all_times = raw_sub_da.time.values
all_naive_dates, naive_analog_dates = get_naive_sample_dates(all_times, ref_date)

`naive_analog_dates` should be contained within the `all_naive_dates` (ok if type is different):

In [105]:
assert np.all([dt in all_naive_dates for dt in naive_analog_dates])

`naive_analog_dates` should not contain dates that are within 15 days of forecast date:

In [106]:
ref_dt = pd.to_datetime(ref_date + " 12:00:00")
forecast_length = 14
between = pd.Series(naive_analog_dates).between(
    ref_dt - pd.to_timedelta(forecast_length + 2, "D"),
    ref_dt + pd.to_timedelta(forecast_length + 1, "D")
)
assert np.all(~between)

`naive_analog_dates` should all be within +/- 45 days of day-of-year of `ref_date`:

In [197]:
ref_day = ref_dt.day
ref_month = ref_dt.month

# define a function to check that the chosen analog is within +/- 45 days of the
#  date corresponding to the same day of year as reference date
def check_analog_in_window(dt, ref_month, ref_day):
    dt = pd.to_datetime(dt)
    year = pd.to_datetime(dt).year
    tmp_ref_dt = pd.to_datetime(f"{year}-{ref_month}-{ref_day} 12:00:00")
    td_offset = pd.to_timedelta(45, "D")
    start_window = tmp_ref_dt - (td_offset + pd.to_timedelta(1, "D"))
    end_window = tmp_ref_dt + td_offset
    test = pd.Series(tmp_ref_dt).between(start_window, end_window)

    return test

# 
assert np.all([get_window_from_analog_date(dt, ref_month, ref_day) for dt in naive_analog_dates])

end